# **Challenge 2**

Luca Vignola, Tommaso Parma, Giorgio Mantoan

**DESCRIZIONE**\
L'agoritmo implementato è il "Worst-case linear-time order staitics" il quale trova l'i-esimo elemento di un vettore di dimensione n.
L'algoritmo può essere schematizzato nel seguente modo:

INPUT: (i,n) - i elemento da trovare, n dimensioni del vettore\
          1. Dividere gli elementi in blocchi di 5 e trovare la mediana di ogni gruppo\
          2. Selezionare come pivot, ricorsivamente, la mediana delle mediane dei gruppi composti da 5 elemnti\
          3. Partizionare attorno al pivot x e imporre k=rank(x)\
          4. if i=k return x\
              elseif i<k\
                  selezionare ricorsivamente i-esimo più piccolo elemento nella parte inferirore\
                else\
                   selezionare ricorsivamente (i-k)-esimo più piccolo elemento nella parte superiore.\
OUTPUT: valore dell'elemento in i-esima pozione

**CONSIDERAZIONI**\
L'algoritmo, se eseguito, fornisce il risultato corretto: in tutte le casistiche affrontate viene fornito l'elemento in i-esima posizione.
In tutte le esecuzioni è stato considerato un array di n elemnti che contenesse i valori da 1 a n. Questo facilita la fase di testing in quanto la risposta corretta della query per l'elemento i sarà proprio i stesso. Per testare abbiamo quindi implementato un for che testa tutti gli i che vanno da 1 ad un n molto grande, e restituisce 1 alla fine del ciclo se tutti i testcase erano corretti, 0 altrimenti. L'algoritmo è stato testato sia nel caso di vettore disordinato casualmente che nel caso ordinato.

Nonostante l'algoritmo funzioni correttamento esso mostra una complessità differente da quella teorica: la complessità effettiva calcolata tramite libreria di benchmarking risulta essere 0.00*O(N^2) contro quella teorica di O(N) (lineare).\
Tale discrepanza può essere attribuita a fattori legati all'implementazione del codice, che potrebbe generare costanti così grandi da essere interpretate come una classe di complessità superiore dall'algoritmo di benchmarking.
Di seguito alcune delle strategie adottate per cercare di abbattere queste costanti, che hanno contribuito ad abbassare il risultato da 0.14*O(N^2) a 0.00*O(N^2) (comunque quadratico).

- Una parte cruciale dell'algoritmo è il calcolo della mediana. Nell'implementazione iniziale restituivamo l'elemento in posizione intermedia del vettore dopo aver eseguito un bubble sort per ordinarlo. Nella versione finale abbiamo implementato un'algoritmo ottimizzato per calcolare la mediana di un vettore di 5 elementi con il numero minimo teorico di operazioni di confronto, pari a 6(gestendo poi i casi di lunghezza inferiore a 5 manualmente)
- abbiamo provato a modificare la dimensione del caso base: Un'idea che abbiamo avuto è che aumentando la dimensione del caso base si potesse guadagnare tempo fermando prima l'albero di ricorsione. Tuttavia questo non ha portato benefici a livello pratico quindi siamo tornati al caso base di 1 elemento.
- la dichiarazione degli std::vector è stata spostata all'esterno dello scope della funzione come variabile globale. La sua dimensione è comunque stata allocata preventivamente tramite il metodo resize, consentendo di evitare le push_back, che per gli std::vector hanno una complessità (ammortizzata!) costante.


# **Notebook setup**

**Download the code**

In [ ]:
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest

Cloning into 'benchmark'...
remote: Enumerating objects: 8559, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 8559 (delta 66), reused 107 (delta 43), pack-reused 8405
Receiving objects: 100% (8559/8559), 2.77 MiB | 13.08 MiB/s, done.
Resolving deltas: 100% (5707/5707), done.
Cloning into 'benchmark/googletest'...
remote: Enumerating objects: 27461, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 27461 (delta 0), reused 4 (delta 0), pack-reused 27455
Receiving objects: 100% (27461/27461), 12.59 MiB | 11.45 MiB/s, done.
Resolving deltas: 100% (20404/20404), done.


# Nuova sezione

**Organize the code and install**

In [ ]:
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Failed to find LLVM FileCheck
-- Found Git: /usr/bin/git (found version "2.34.1") 
-- Google Benchmark version: v1.8.3-73-gbc946b91, normalized to 1.8.3.73
-- Looking for shm_open in rt
-- Looking for shm_open in rt - found
-- Performing Test HAVE_CXX_FLAG_WALL
-- Performing Test HAVE_CXX_FLAG_WALL - Success
-- Performing Test HAVE_CXX_FLAG_WEXTRA
-- Performing Test HAVE_CXX_FLAG_WEXTRA - Success
-- Performing Test HAVE_CXX_FLAG_WSHADOW
-- Performing Test HAVE_CXX_FLAG_WSHADOW - Success
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL - Success
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST - Success
-- Performing Test HAVE_CXX_FLAG_WCO

# **Challenge 2**

In [ ]:
%%writefile challenge.cpp

#include <benchmark/benchmark.h>
#include <ctime>
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm>
#include <random>
std::vector <int> mediane;

int min(int a, int b){
    if(a<b) return a;
    return b;
}

int calc_mediana(std::vector <int> &v, int start, int end) {

  int a, b, c, d;
  switch (end-start){
    case 1:
      return v[start];
    break;
    case 2:
      return min(v[start], v[start+1]);
    break;
    case 3:
      if(v[start] < v[start+1]){
        if(v[start+1] < v[start+2]) return v[start+1];
        else if(v[start+2] < v[start]) return v[start];
        else return v[start+2];
      } else if(v[start+1] < v[start+2]) return min(v[start], v[start+2]);
      else return v[start+1];
    break;
    case 4:

      if(v[start] > v[start+1]) {
        int temp = v[start];
        v[start] = v[start+1];
        v[start+1] = temp;
      }
      if(v[start+2] > v[start+3]){
        int temp = v[start+2];
        v[start+2] = v[start+3];
        v[start+3] = temp;
      }
    if(v[start] < v[start+2]) return min(v[start+2], v[start+1]);
    else return min(v[start+3], v[start]);
    break;


    case 5:
      if(v[start] < v[start+1]) {
        a = v[start];
        b = v[start+1];
      } else {
        a = v[start+1];
        b = v[start];
      }
      if(v[start+2] < v[start+3]){
        c = v[start+2];
        d = v[start+3];
      } else {
        c = v[start+3];
        d = v[start+2];
      }
      if(a < c){
        a = v[start+4];
        if(a > b)
        {
          int temp = a;
          a = b;
          b = temp;
        }
      }
      else{
        c = v[start+4];
        if(c > d){
          int temp = c;
          c = d;
          d = temp;
        }
      }
      if(a < c){
        if(b<c) return b;
        else return c;
      } else{
        if(a<d) return a;
        else return d;
      }
    break;
}

   for(int j=start;j<end-1;j++)
		for(int i=start;i<end-1;i++)
			if(v[i]>v[i+1])
			{
				int temp=v[i];
				v[i]=v[i+1];
				v[i+1]=temp;
			}
    return v[start + (end-start-1)/2];


}
int select(std::vector<int> v, int i, int start, int end) {
  if(end-start <= 1) {
    /*for(int j=start;j<end-1;j++)
		for(int k=start;k<end-1;k++)
			if(v[k]>v[k+1])
			{
				int temp=v[k];
				v[k]=v[k+1];
				v[k+1]=temp;
			}*/
    return v[start+i-1];
  }
  int nblocchi;
  int last = (end-start)%5;
  if(last % 5 == 0) nblocchi = ceil((end-start)/5);
  else{
   nblocchi = ceil((end-start)/5)+1;
  }
  for(int j = start; j < end; j += 5) {
    mediane[(j-start)/5] = calc_mediana(v, j, min(j+5,end));
  }




  int med = nblocchi/2 + nblocchi%2;
  int x = select(mediane, med, 0, nblocchi);

  int pivot = x;
  int a = start;
  for(int b = start+1; b < end; b++) {
    if(v[b] == pivot) {
      v[b] = v[start];
      v[start] = pivot;
      b--;
    } else if(v[b] < pivot) {
      a = a+1;
      int temp = v[a];
      v[a] = v[b];
      v[b] = temp;
    }
  }
  v[start] = v[a];
  v[a] = pivot;

  int k = a;

  k = k-start;
  if(k+1 == i) return x;
  else if(i < k+1) return select(v, i, start, start+k);
  else return select(v, i-k-1, start+k+1, end);
}

static void BM_select(benchmark::State& state)
{
  int size=state.range(0);
  std::srand(unsigned(std::time(nullptr)));
  std::vector<int> v(size);
  std::generate(v.begin(), v.end(), std::rand);
  int i = rand()%size + 1;
  mediane.resize(size/5 + 1);
  for (auto _ :state)
        benchmark::DoNotOptimize(select(v, i, 0, size));
    state.SetComplexityN(state.range(0));
}

BENCHMARK(BM_select)
    ->RangeMultiplier(2)
    ->Range(1<<2, 1<<18)
    ->Complexity();


BENCHMARK_MAIN();

/*
int main() {
  int n = 300;
  std::vector<int> v(n);
  std::generate(v.begin(), v.end(), std::rand);
  mediane.resize(n/5 + 1);
  for(int i = 0; i < n; i++) v[i] = (i+1);


  //testa un vettore di lunghezza n contenente i numeri da 1 a n: Il risultato teorico è i per la ricerca dell'iesimo elemento quindi se il cout finale è 1 so che sono tutti corretti
  int result = 1;
  for(int i = 1; i <= n; i++){
    std::shuffle(v.begin(), v.end(), std::default_random_engine(5));
    //for(int l = 0; l < n; l++) std::cout<<v[l]<<" ";
    //std::cout<<" "<<std::endl<<std::endl;
    if(i != select(v, i, 0, n)) result = 0;
  }
  std::cout<<"\n\n"<<result;


}
*/


Writing challenge.cpp


In [ ]:
!g++ challenge.cpp -O2 -std=c++11 -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o challenge2

In [ ]:
!./challenge2

2024-04-28T09:43:32+00:00
Running ./challenge2
Run on (2 X 2200 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 1.26, 0.97, 0.47
-----------------------------------------------------------
Benchmark                 Time             CPU   Iterations
-----------------------------------------------------------
BM_select/4             141 ns          139 ns      5045552
BM_select/8             190 ns          189 ns      3765975
BM_select/16            342 ns          339 ns      2151096
BM_select/32            498 ns          495 ns      1312082
BM_select/64           1342 ns         1332 ns       533086
BM_select/128          1428 ns         1419 ns       506641
BM_select/256          4719 ns         4692 ns       174795
BM_select/512          8673 ns         8602 ns        85618
BM_select/1024        38917 ns        37873 ns        24288
BM_select/2048       127728 ns       122251 ns         56